In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt


In [2]:
from cvxopt import matrix, solvers

In [4]:
!wget http://www.amlbook.com/data/zip/features.train

--2017-11-14 20:49:26--  http://www.amlbook.com/data/zip/features.train
Resolving www.amlbook.com (www.amlbook.com)... 118.139.174.1
Connecting to www.amlbook.com (www.amlbook.com)|118.139.174.1|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 357259 (349K)
Saving to: ‘features.train’

features.train      100%[===================>] 348.89K  2.00MB/s    in 0.2s    

2017-11-14 20:49:26 (2.00 MB/s) - ‘features.train’ saved [357259/357259]



In [6]:
!wget http://www.amlbook.com/data/zip/features.test

--2017-11-14 20:50:11--  http://www.amlbook.com/data/zip/features.test
Resolving www.amlbook.com (www.amlbook.com)... 118.139.174.1
Connecting to www.amlbook.com (www.amlbook.com)|118.139.174.1|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 98343 (96K)
Saving to: ‘features.test’

features.test       100%[===================>]  96.04K  --.-KB/s    in 0.06s   

2017-11-14 20:50:20 (1.67 MB/s) - ‘features.test’ saved [98343/98343]



In [2]:
data_train = np.loadtxt('features.train')
data_test = np.loadtxt('features.test')

data_train.shape, data_test.shape

((7291, 3), (2007, 3))

In [16]:
data_test[:5]

array([[ 9.        ,  0.27217773, -4.8479375 ],
       [ 6.        ,  0.26513281, -5.102     ],
       [ 3.        ,  0.33592578, -2.9215625 ],
       [ 6.        ,  0.26484961, -4.156625  ],
       [ 6.        ,  0.34533789, -6.7184375 ]])

In [17]:
data_train[:5]

array([[ 6.        ,  0.3410918 , -4.5289375 ],
       [ 5.        ,  0.44413086, -5.4968125 ],
       [ 4.        ,  0.23100195, -2.88675   ],
       [ 7.        ,  0.20027539, -3.534375  ],
       [ 3.        ,  0.29193555, -4.3520625 ]])

In [3]:
X_train, y_train = data_train[:, 1:], data_train[:, 0]
X_test, y_test = data_test[:, 1:], data_test[:, 0]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((7291, 2), (7291,), (2007, 2), (2007,))

In [20]:
pd.value_counts(y_train) / len(y_train) * 100

0.0    16.376354
1.0    13.784117
2.0    10.026060
6.0     9.107118
3.0     9.024825
4.0     8.942532
7.0     8.846523
9.0     8.832808
5.0     7.625840
8.0     7.433823
dtype: float64

In [21]:
pd.value_counts(y_test) / len(y_test) * 100

0.0    17.887394
1.0    13.153961
4.0     9.965122
2.0     9.865471
9.0     8.819133
6.0     8.470354
8.0     8.271051
3.0     8.271051
5.0     7.972098
7.0     7.324365
dtype: float64

In [24]:
#solvers.options['show_progress'] = False

### SVM with Soft Margins

In [4]:
def poly_kernel(x1, x2, q=2):
    '''polynomial kernel : (1 + dot(x1,x2))**q'''
    return (1 + np.dot(x1, x2))**q


def rbf_kernel(x1, x2, gamma=1):
    '''radial basis function : e**(-gamma*||x1 - x2||)'''
    return np.exp(-gamma * np.linalg.norm(x1 - x2)**2)


class MySVM:
    '''soft margin SVM'''
    def __init__(self, C=None, kernel=np.dot):
        self.C = C
        self.kernel = kernel
        
    def fit(self, X, y):
        from cvxopt import matrix, solvers
        #solvers.options['show_progress'] = False
        
        n, m = X.shape
        
        # create Gram matrix
        P = np.empty((n, n))
        for i in range(n-1):
            P[i, i] = self.kernel(X[i], X[i])
            for j in range(i+1, n):
                P[i, j] = P[j, i] = self.kernel(X[i], X[j])
        P[i+1, i+1] = self.kernel(X[i+1], X[i+1])
        P = matrix(P)
        
        # -a = 0
        q = np.ones(n) * -1.0
        q = matrix(q)
        
        # Y*a = 0
        A = matrix(np.diag(y), tc='d')
        b = matrix(np.zeros(n))
        
        # -a <= 0
        if self.C is None:
            # hard margin SVM
            G = matrix(-np.eye(n, dtype=float))
            h = matrix(np.zeros(n))
        # if C : a <= C, -a <= 0
        else:
            # soft margin SVM
            G = matrix(np.vstack((-np.eye(n, dtype=float), np.eye(n, dtype=float))))
            h = matrix(np.r_[np.zeros(n), np.full(n, self.C)])
        
        # qp(P, q, G, h, A, b)
        alphas = np.ravel(solvers.qp(P, q, G, h, A, b)['x'])
        
        is_svm = (alphas > 1e-5)
        print(alphas.min())
        assert is_svm.sum() > 0
        self.alphas_ = alphas[is_svm]
        self.y_svm_ = y[is_svm]
        self.support_vectors_ = X[is_svm, :]
        print(self.alphas_.shape, self.y_svm_.shape, self.support_vectors_.shape)
        self.b_ = self.y_svm_[0] - sum(y*a*self.kernel(x, self.support_vectors_[0, :]) 
                                                       for y,a,x in zip(self.y_svm_, self.alphas_, self.support_vectors_))
        return self
        
    def predict(self, X):
        results = np.empty(len(X))
        for i, xs in enumerate(X):
            results[i] = self.b_ + sum(a*y*self.kernel(x, xs) 
                                       for a, y, x in zip(self.alphas_, self.y_svm_, self.support_vectors_))
        return results

In [9]:
from sklearn.svm import SVC

In [43]:
L = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
for num in L:
    y_vs_all = np.where(y_train==num, 1, -1)
    svm = SVC(C=0.01, kernel='poly', gamma=1, coef0=1, degree=2).fit(X_train, y_vs_all)
    print('E_in[{}] = {}, {} support vectors'.format(num, 1-svm.score(X_train, y_vs_all), svm.n_support_.sum()))

E_in[0] = 0.10588396653408316, 2179 support vectors
E_in[1] = 0.014401316691811772, 386 support vectors
E_in[2] = 0.10026059525442321, 1970 support vectors
E_in[3] = 0.09024825126868741, 1950 support vectors
E_in[4] = 0.08942531888629812, 1856 support vectors
E_in[5] = 0.07625840076807022, 1585 support vectors
E_in[6] = 0.09107118365107669, 1893 support vectors
E_in[7] = 0.08846523110684401, 1704 support vectors
E_in[8] = 0.074338225209162, 1776 support vectors
E_in[9] = 0.08832807570977919, 1978 support vectors


In [45]:
2179 - 386

1793

In [31]:
one_or_five_train = (y_train == 1) | (y_train == 5)
one_or_five_test = (y_test == 1) | (y_test == 5)

In [34]:
X_train_1_vs_5 = X_train[one_or_five_train, :]
y_train_1_vs_5 = np.where(y_train[one_or_five_train]==1, 1, -1)

X_test_1_vs_5 = X_test[one_or_five_test, :]
y_test_1_vs_5 = np.where(y_test[one_or_five_test]==1, 1, -1)

In [44]:
for C in [.0001, .001, .01, .1, 1]:
    for Q in [2, 5]:
        svm = SVC(C=C, kernel='poly', gamma=1, degree=Q, coef0=1).fit(X_train_1_vs_5, y_train_1_vs_5)
        print("C={}, Q={}, E_in:{:.5f}, E_out:{:.5f}, num_SV={}".format(C, Q,
                                                      1-svm.score(X_train_1_vs_5, y_train_1_vs_5),
                                                      1-svm.score(X_test_1_vs_5, y_test_1_vs_5), 
                                                      svm.n_support_.sum()))

C=0.0001, Q=2, E_in:0.00897, E_out:0.01651, num_SV=236
C=0.0001, Q=5, E_in:0.00448, E_out:0.01887, num_SV=26
C=0.001, Q=2, E_in:0.00448, E_out:0.01651, num_SV=76
C=0.001, Q=5, E_in:0.00448, E_out:0.02123, num_SV=25
C=0.01, Q=2, E_in:0.00448, E_out:0.01887, num_SV=34
C=0.01, Q=5, E_in:0.00384, E_out:0.02123, num_SV=23
C=0.1, Q=2, E_in:0.00448, E_out:0.01887, num_SV=24
C=0.1, Q=5, E_in:0.00320, E_out:0.01887, num_SV=25
C=1, Q=2, E_in:0.00320, E_out:0.01887, num_SV=24
C=1, Q=5, E_in:0.00320, E_out:0.02123, num_SV=21


### Cross Validation

In [46]:
from sklearn.model_selection import StratifiedKFold

In [77]:
C_list = [.0001, .001, .01, .1, 1.]
cv = StratifiedKFold(n_splits=10, shuffle=True)
C_selected = {c: 0 for c in C_list}
E_cv_avg = {c: 0 for c in C_list}

for run in range(100):
    E_cv_by_C = {c: 0 for c in C_list}
    for train_idx, test_idx in cv.split(X_train_1_vs_5, y_train_1_vs_5, y_train_1_vs_5):
        X_train_cv = X_train_1_vs_5[train_idx, :]
        y_train_cv = y_train_1_vs_5[train_idx]
        X_test_cv = X_train_1_vs_5[test_idx, :]
        y_test_cv = y_train_1_vs_5[test_idx]
        for c in C_list:
            svm = SVC(C=c, kernel='poly', gamma=1, coef0=1, degree=2).fit(X_train_cv, y_train_cv)
            E_cv_by_C[c] += 1-svm.score(X_test_cv, y_test_cv)
    c_min = min(((v,k) for k,v in E_cv_by_C.items()), key=lambda x: x[0])[1]
    for c, e in E_cv_by_C.items():
        E_cv_avg[c] += e/10
        
    C_selected[c_min] += 1

In [78]:
C_selected

{0.0001: 0, 0.001: 49, 0.01: 23, 0.1: 14, 1.0: 14}

In [79]:
{k:v/100 for k,v in E_cv_avg.items()}

{0.0001: 0.0097563196409097448,
 0.001: 0.0047601607898299951,
 0.01: 0.0046956451876320419,
 0.1: 0.004760037247185391,
 1.0: 0.004881754674337377}

### RBF Kernel

In [62]:
C_list = .01, 1, 100, 1e4, 1e6

for c in C_list:
    svm = SVC(C=c, kernel='rbf', gamma=1).fit(X_train_1_vs_5, y_train_1_vs_5)
    print("C={}, E_in:{}, E_out:{}".format(c, 
                                           1-svm.score(X_train_1_vs_5, y_train_1_vs_5), 
                                           1-svm.score(X_test_1_vs_5, y_test_1_vs_5)))

C=0.01, E_in:0.0038436899423446302, E_out:0.02358490566037741
C=1, E_in:0.004484304932735439, E_out:0.021226415094339646
C=100, E_in:0.0032030749519538215, E_out:0.018867924528301883
C=10000.0, E_in:0.002562459961563124, E_out:0.02358490566037741
C=1000000.0, E_in:0.0006406149903908087, E_out:0.02358490566037741
